<a href="https://colab.research.google.com/github/balananujith/mulearn-ai-isitabird/blob/master/is_it_a_bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries from fastai for computer vision tasks
from fastai.vision.all import *

# Import the Path class from pathlib for filesystem path manipulations
from pathlib import Path

# Import sleep function from time to introduce delays in the code execution
from time import sleep

# Import download_url from fastdownload to facilitate downloading files
from fastdownload import download_url

# Install the required package for performing searches using DuckDuckGo
!pip install duckduckgo_search

# Import the DuckDuckGo search functionality
from duckduckgo_search import DDGS

# Import utility functions from fastcore for additional functionality
from fastcore.all import *


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 12.3 MB/s eta 0:00:00


In [3]:
def search_images(term, max_images=30):
    # Print the search term to the console
    print(f"Searching for '{term}'")

    # Use DuckDuckGo search within a context manager to ensure proper resource management
    with DDGS() as ddgs:
        # Perform an image search with the given keywords
        # This returns a generator which yields dictionaries containing:
        # {'title', 'image', 'thumbnail', 'url', 'height', 'width', 'source'}
        search_results = ddgs.images(keywords=term)

        # Extract the URLs of the images from the search results
        # Use a list comprehension to get the 'image' field from each result
        # Limit the number of URLs to max_images
        image_urls = [next(iter(search_results)).get("image") for _ in range(max_images)]

        # Convert the list of image URLs to an L object (a functionally extended list class from fastai)
        return L(image_urls)


In [4]:
# Define search categories
searches = ('forest', 'bird')

# Define the main folder path
path = Path('bird_or_not')

# Loop through each category in the search categories
for category in searches:
    # Form sub-folder path based on category
    dest = path / category

    # Create the sub-folder if it doesn't already exist
    dest.mkdir(exist_ok=True, parents=True)

    # Print message indicating the category being downloaded
    print(f"Downloading images for category: {category}")

    # Download images for the category with specific search terms
    download_images(dest, urls=search_images(f'{category} photo'))
    sleep(10)  # Pause between searches to avoid overloading the server

    download_images(dest, urls=search_images(f'{category} sun photo'))
    sleep(10)  # Pause between searches to avoid overloading the server

    download_images(dest, urls=search_images(f'{category} shade photo'))
    sleep(10)  # Pause between searches to avoid overloading the server

    # Resize images in the category sub-folder to a maximum size of 400 pixels
    resize_images(path/category, max_size=400, dest=path/category)

# Verify downloaded images and identify failed images
failed = verify_images(get_image_files(path))

# Remove failed images
failed.map(Path.unlink)  # Unlink (delete) the failed images



Searching for 'forest photo'
Searching for 'forest sun photo'
Searching for 'forest shade photo'
Searching for 'bird photo'
Searching for 'bird sun photo'
Searching for 'bird shade photo'


(#0) []

In [5]:
# Prepare data for training
dls = DataBlock(
    # Define the type of data (image and category for classification)
    blocks=(ImageBlock, CategoryBlock),

    # Specify how to get the items (images) from the path
    get_items=get_image_files,

    # Split the data into training and validation sets (80% training, 20% validation)
    splitter=RandomSplitter(valid_pct=0.2, seed=42),

    # Define how to get the labels (categories) for each item (image)
    get_y=parent_label,

    # Apply transformation to resize images to 192x192 pixels using the 'squish' method
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)  # Create dataloaders with batch size of 32

# Create and fine-tune the image classification model
learn = vision_learner(dls, resnet18, metrics=error_rate)  # Use a pre-trained resnet18 model
learn.fine_tune(3)  # Fine-tune the model for 3 epochs

# Make a prediction on a new image
urls = search_images('forest photos', max_images=1)  # Search for a new image with the term 'forest photos'
dest = 'forest.jpg'  # Define the destination path for the downloaded image
download_url(urls[0], dest, show_progress=False)  # Download the image without showing progress
im = Image.open(dest)  # Open the downloaded image
im.to_thumb(256,256)  # Convert the image to a thumbnail of size 256x256 pixels

# Predict the class of the new image using the trained model
is_bird, _, probs = learn.predict(PILImage.create("forest.jpg"))
print(f"This is a: {is_bird}.")  # Print the predicted class
print(f"Probability it's a bird: {probs[0]:.4f}")  # Print the probability that the image is of a bird


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 85.9MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,0.711692,0.008610,0.000000,00:22


epoch,train_loss,valid_loss,error_rate,time
0,0.000742,0.000049,0.000000,00:35
1,0.000409,0.000000,0.000000,00:33
2,0.000267,0.000000,0.000000,00:36


Searching for 'forest photos'


This is a: forest.
Probability it's a bird: 0.0000
